In [ ]:
import sys

sys.path.append("../")

from torch.utils.data import DataLoader

from src.data import PeakWeatherTorchDataset, test_model, plot_predictions
from src.model import MoiraiModel

### Scegliere la finestra di osservazione e l'orizzonte di previsione
Vogliamo prevedere le prossime 24 ore, come con i modelli ARIMA e MLP.

I modelli di deep learning fondazionali spesso richiedono finestre di osservazioni ancora più lunghe per catturare le dinamiche temporali.
Per ora usiamo una finestra di 48 ore (2 giorni), come fatto per il modello MLP.

In [ ]:
horizon = (
    24  # Lasciamo l'orizzonte di previsione a 24 ore per confrontare con ARIMA e MLP
)
window = 48

### Prendiamo il test set
Creiamo il test set usato per valutare le prestazioni del modello dopo l'addestramento.

#### Domanda
Noti qualche differenza con quanto fatto per MLP?

In [ ]:
test_dataset = PeakWeatherTorchDataset(window="???", horizon="???")
test_dataset.mode = "test"

# Create DataLoaders
batch_size = 8192
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Carichiamo il modello
Creiamo il modello Moirai.

Nota come, in questo caso, non abbiamo bisogno di addestrare il modello, in quanto stiamo caricando un modello pre-addestrato da qualcun'altro per noi.

#### Domanda
Cosa implica il fatto che il modello sia pre-addestrato? (Suggerimento: cosa hanno visto gli altri modelli che questo modello non ha visto?)

In [ ]:
model = MoiraiModel(window="???", horizon="???")

### Testiamo il modello sui dati di test
Calcoliamo l'errore medio assoluto (MAE) del modello sui dati di test.

#### Domande
- Come si confronta questo valore con quello ottenuto con il modello ARIMA?
- Come si confronta questo valore con quello ottenuto con il modello MLP?
- In ognuno dei due casi, i due valori sono direttamente confrontabili? Perché sì o perché no?

In [ ]:
mae = test_model(model="???", test_loader=test_dataloader)

### Visualizziamo alcune previsioni del modello
Tracciamo alcune previsioni del modello sui dati di test per valutarne le prestazioni visivamente.

Puoi scambiare il numero di campioni visualizzati modificando il parametro `num_samples`.

In [ ]:
plot_predictions(model="???", test_dataset=test_dataset, num_samples=5)

#### Esercizio
- Prova a variare il numero di osservazioni in input (window) e osserva come cambiano le prestazioni del modello.
- Cerca di migliorare il MAE ottenuto e di ottenere previsioni più accurate.
- Documenta le tue modifiche e i risultati ottenuti.

**Consiglio**: Non eccedere con la lunghezza della finestra. Un buon limite è:
- window ≤ 720